In [1]:
import pandas as pd
import os
import openai
import json
import codecs
from sklearn.model_selection import StratifiedShuffleSplit
openai.api_key = "sk-W2v63UWK3U2J1CzRVFoAVhwIZkQd7mS4fqfrvd9E"

In [2]:
tot_file = 'data/raw/text_extraction.json'
targets = 'data/raw/target_values.json'
with open(tot_file) as f:
    data_tot = json.load(f)
    
with open(targets) as f:
    data_target = json.load(f)
    
clean_data_tot = {}
for el in data_tot:
    clean_data_tot[el['complete_filename']] = el

In [6]:
def create_df(data_tot, data_target):
    names, cause, keywords, texts = [], [], [], []
    for file_name in data_target:
        names.append(file_name)
        els = data_target[file_name]
        cause.append(els[0][0])
        keywords.append(';'.join(els[1]))
        texts.append(clean_data_tot[file_name]['text'])
    train_df = pd.DataFrame({'file_name':names, 'cause':cause, 'keywords':keywords, 'text':texts})
    
    names, texts = [], []
    for file_name in data_tot:
        if file_name not in data_target:
            names.append(file_name)
            texts.append(clean_data_tot[file_name]['text'])
    test_df = pd.DataFrame({'file_name':names, 'text':texts})
    return train_df, test_df

train_df, test_df = create_df(clean_data_tot, data_target)
train_df.head()

,file_name,cause,keywords,text
0,0901dbf5901f8d4f_1_2016000033828800.PDF,vento,temporale,PR Servizi Assicurativi srl 24128 BERGAMO Œ ...
1,0901dbf5953381cf_3_2018000011862400.PDF,vento,pioggia;grandine;evento atmosferico;fortunale,ˇ ˆ ˇ ˇ ˆ ˆ+...
2,0901dbf59743d16a_3_2019000011916400.PDF,temporale,scariche elettriche,ˇˆˆˆˆ ˆˆˆ+ˆˆ-ˆˆ/ˆˆ+ˆˆˇˇˆ ˆˆ0ˆ 12ˆ0ˆˆ32ˆ432ˆ32ˆ...
3,0901dbf59b271fd7_1_2020000064464100.PDF,temporale,evento atmosferico,17468-2020/DOMilano22/12/2020CompagniaPzzaArma...
4,0901dbf59209a167_3_2017000010254900.PDF,grandine,evento atmosferico,ˇˇˆˆˆˆˆˆˆˆ ˆˆ++--//ˇˆˆˆ ˇˆˆ+-+- 0/0ˆ112+2-...


In [7]:
test_df.head()

,file_name,text
0,0901dbf58e3b1554_3_2016000000964100.PDF,...
1,0901dbf58e5ef1f8_3_2016000002199700.PDF,Compagnia assicurativaITALIANA ASSICURAZIONICe...
2,0901dbf58e607d02_1_2016000006186200.PDF,Studio RUSSO SrlPerizie - Consulenza e Tecnic...
3,0901dbf58e63ef8d_3_2016000001266300.PDF,L A SPE Z IA se d e lega le P Iva 01 114 56...
4,0901dbf58e65ef7f_3_2016000002269000.PDF,SpettleNostro riferimentoSinistro nAssicuratoA...


In [8]:
id_columns = ['file_name']
input_columns = ['text']
label_columns = ['keywords','cause']
def df_to_json(df):
    output_list = []
    values = df[id_columns+input_columns+label_columns].values
    for value in values:
        output_dict = {"prompt":"","completion":""}
        c = 0
        for col in id_columns:
            output_dict["prompt"] += col + ': ' +str(value[c])+' # '
            c+=1
        for col in input_columns:
            output_dict["prompt"] += col + ': ' +str(value[c])+' # '
            c+=1
        for col in label_columns:
            output_dict["completion"] += col + ': ' +str(value[c])+' # '
            c+=1
        output_dict["completion"] = output_dict["completion"][:-3]+ ' ###'
        output_list.append(output_dict)
    return output_list

with open('data/processed/file.jsonl', 'w', encoding='UTF-8') as f:
    for el in df_to_json(train_df):
        f.write(json.dumps(el)+'\n')

In [37]:
openai.File.create(
  file=open("data/processed/file.jsonl"),
  purpose='fine-tune'
)

<File file id=file-dfUO6kOUudUU2xar7cUhWPJ7 at 0x1fcad08fd68> JSON: {
  "bytes": 481752,
  "created_at": 1653471972,
  "filename": "file.jsonl",
  "id": "file-dfUO6kOUudUU2xar7cUhWPJ7",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [38]:
openai.FineTune.create(training_file="file-dfUO6kOUudUU2xar7cUhWPJ7")

<FineTune fine-tune id=ft-2MI16U2A1xK95O7aHDSxW9rQ at 0x1fcadb0a8e8> JSON: {
  "created_at": 1653471983,
  "events": [
    {
      "created_at": 1653471983,
      "level": "info",
      "message": "Created fine-tune: ft-2MI16U2A1xK95O7aHDSxW9rQ",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-2MI16U2A1xK95O7aHDSxW9rQ",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-sAS7XXyTNjHCesZhGqONi6b9",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 481752,
      "created_at": 1653471972,
      "filename": "file.jsonl",
      "id": "file-dfUO6kOUudUU2xar7cUhWPJ7",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1653471983,
  "validation_files": []
}

In [40]:
openai.FineTune.retrieve(id="ft-2MI16U2A1xK95O7aHDSxW9rQ")

<FineTune fine-tune id=ft-2MI16U2A1xK95O7aHDSxW9rQ at 0x1fcab4c3c48> JSON: {
  "created_at": 1653471983,
  "events": [
    {
      "created_at": 1653471983,
      "level": "info",
      "message": "Created fine-tune: ft-2MI16U2A1xK95O7aHDSxW9rQ",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1653472000,
      "level": "info",
      "message": "Fine-tune costs $2.50",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1653472000,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1653472002,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1653472096,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1653472142,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune-

In [6]:
tdf = test_df
tdf[label_columns] = ''
inp = df_to_json(tdf.iloc[6:16])

In [8]:
model = 'curie:ft-personal-2022-05-25-09-50-58'

def predict(inp, model):
    responses = []
    for el in inp:
        prompt = el['prompt'][:3890]
        if not prompt.endswith(' #'):
            prompt = prompt +' #'
        res = openai.Completion.create(
                      model=model,
                      prompt=prompt,
                      max_tokens=100
                    )
        text = res['choices'][0]['text'].split(' ###')[0]
        responses.append(text)
    return responses

In [9]:
resps = predict(inp, model)

In [10]:
resps

[' keywords: evento atmosferico # cause: vento',
 ' keywords: eventi atmosferici;sovratensione # cause: fulmine',
 ' keywords: tromba daria # cause: vento',
 ' keywords: evento atmosferico;fortunale;acqua piovano # cause: vento',
 ' keywords: tromba # cause: vento',
 ' keywords: evento atmosferico # cause: vento',
 ' keywords: eventi atmosferici;sovratensione # cause: fulmine',
 ' keywords: danno occasionato # cause: vento',
 ' keywords: sovra tensione # cause: fenomeno elettrico',
 ' keywords: vento # cause: grandine']

In [11]:
list(zip(list(map(lambda x: x['prompt'].split(' #')[0],inp)), resps))

[('file_name: 0901dbf58e82947f_1_2016000006841700.PDF',
  ' keywords: evento atmosferico # cause: vento'),
 ('file_name: 0901dbf58e876ddb_1_2016000004718900.PDF',
  ' keywords: eventi atmosferici;sovratensione # cause: fulmine'),
 ('file_name: 0901dbf58e8d5a61_3_2016000003897500.PDF',
  ' keywords: tromba daria # cause: vento'),
 ('file_name: 0901dbf58e8fcc25_3_2016000002938200.PDF',
  ' keywords: evento atmosferico;fortunale;acqua piovano # cause: vento'),
 ('file_name: 0901dbf58e8fd640_1_2016000006614000.PDF',
  ' keywords: tromba # cause: vento'),
 ('file_name: 0901dbf58e905993_1_2016000010259000.PDF',
  ' keywords: evento atmosferico # cause: vento'),
 ('file_name: 0901dbf58e9b372c_3_2016000002959700.PDF',
  ' keywords: eventi atmosferici;sovratensione # cause: fulmine'),
 ('file_name: 0901dbf58e9bd60b_1_2016000010735000.PDF',
  ' keywords: danno occasionato # cause: vento'),
 ('file_name: 0901dbf58e9faed7_1_2016000004788200.PDF',
  ' keywords: sovra tensione # cause: fenomeno elet